<a href="https://colab.research.google.com/github/hesolar/TrabajoDistribuidos2020solar-escudero/blob/master/practica9_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 9 Parte 1: Word embeddings

En este primer notebook veremos distintos word embeddings. Para ello usaremos una librería de procesamiento de lenguaje natural llamada [gensim](https://radimrehurek.com/gensim/index.html). 

## Modelo antiguo

Antes de hablar de los modelos modernos, conviene conocer los modelos antiguos y sus limitaciones. 

Una de las técnicas más utilizadas hasta hace poco para codificar frases era conocida como [bolsa de palabras (o *bag of words*)](https://en.wikipedia.org/wiki/Bag-of-words_model). Esta técnica transforma cada documento a un vector de enteros de longitud fija. 

Por ejemplo, para las siguientes dos frases:

1. A Juan le gusta ver películas. A María también le gusta.
2. A Juan le gusta ver partidos de fútbol. María odia el futbol. 

El modelo produce los siguientes vectores (cada fila de la siguiente tabla representa una de las frases).


| Palabras | A | Juan | le | gusta | ver | películas | María | también | partidos | fútbol | odia | el |
|---------|--------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- |
|Frase 1|2 | 1 | 2 | 1 | 1 | 1 | 1 |1 | 0 | 0 | 0 | 0 |
|Frase 2|1 | 1 | 1 | 1 | 1 | 0 | 1 |0 | 1 | 1 | 1 | 1 |

Cada vector tiene 12 elementos, donde cada elemento cuenta el número de veces que una determinada palabra ocurre en dicho documento. 

Este modelo era muy efectivo, pero tenía varias limitaciones. En primer lugar se pierde información sobre el orden de las palabras. Por ejemplo, las frases "A María le gusta Juan" y "A Juan le gusta María" tienen vectores idénticos. La solución consiste en usar [n-gramas](https://en.wikipedia.org/wiki/N-gram). Además este modelo tiene la limitación de ser *sparse* (muchos ceros para cada vector) y tener una dimensionalidad muy alta. 

Otro de los problemas más importantes es que este modelo no aprende el significado de las palabras subyacentes, y como consecuencia la distancia entre vectores no refleja la similitud o diferencia en significado. Estos problemas se han resuelto con los word-embeddings.





## Word2Vec

El modelo [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf) fue introducido en 2013 por investigadores de Google, y es capaz de incrustar las palabras en un vector de dimensión bja usando una red neuronal. El resultado es un modelo donde los vectores que están cerca en el espacio tienen significados similares basados en el contexto. Es decir, no existe un único modelo word2vec, sino que es un modelo que se entrena a partir de un dataset (dicho dataset puede ser la wikipedia, twitter, ...).

Vamos a ver que es posible hacer con uno de estos modelos. Para ello vamos a usar un modelo entrenado en el dataset de noticias de Google. Por el momento vamos a usar modelos entrenados en inglés, más adelante en la práctica veremos cómo usar modelos en español. 

Comenzamos descargando el dataset (esto puede costar un tiempo). 

In [1]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


Una tarea común es obtener las primeras palabras del vocabulario usado para construir el modelo. 

In [2]:
for i, word in enumerate(wv.vocab):
    if i == 10:
        break
    print(word)

</s>
in
for
that
is
on
##
The
with
said


Ahora podemos ver cuál es la representación de una palabra. 

In [3]:
wv['king']

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

Desafortunadamente el modelo no es capaz de inferir el vector asociado a palabras "raras". 

In [4]:
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

The word 'cameroon' does not appear in this model


Estos modelos proporcionan distintas funciones para tratar varios problemas. 

Por ejemplo, podemos ver la similaridad entre palabras. 


In [5]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


Podemos también mostrar las 5 palabras más similares a *car*.

In [6]:
print(wv.most_similar(positive=['car'], topn=5))

[('vehicle', 0.7821096181869507), ('cars', 0.7423830032348633), ('SUV', 0.7160962820053101), ('minivan', 0.6907036304473877), ('truck', 0.6735789775848389)]


**Ejercicio**: Busca las 5 palabras más similares para Spain. 

In [7]:
print(wv.most_similar(positive=['Spain'], topn=5))

[('Portugal', 0.7220357656478882), ('Inveravante_Inversiones_SL', 0.6925067901611328), ('Spains', 0.6856307983398438), ('Madrid', 0.6743447780609131), ('Spaniards', 0.6629219651222229)]


Es posible también buscar analogías. Por ejemplo, para resolver la analogía *man is to king, as woman is to ...*, se debe ejecutar la siguiente instrucción.

In [8]:
wv.most_similar_cosmul(positive=['king','woman'],negative=['man'])

[('queen', 0.9314123392105103),
 ('monarch', 0.858533501625061),
 ('princess', 0.8476566076278687),
 ('Queen_Consort', 0.8150269985198975),
 ('queens', 0.8099815249443054),
 ('crown_prince', 0.808997631072998),
 ('royal_palace', 0.8027306795120239),
 ('monarchy', 0.801961362361908),
 ('prince', 0.800979733467102),
 ('empress', 0.7958388328552246)]

**Ejercicio**: Da respuesta a las siguientes analogías. 

*Eat is to ate, as go is to ...*

In [11]:
wv.most_similar_cosmul(positive=['ate','go'],negative=['Eat'])

[('went', 0.9246009588241577),
 ('gone', 0.9164908528327942),
 ('stayed', 0.875474214553833),
 ('knew', 0.8727962970733643),
 ('come', 0.8354476690292358),
 ('belonged', 0.8280059695243835),
 ("'d", 0.821956217288971),
 ('slept', 0.8116913437843323),
 ('sneak', 0.8094869256019592),
 ('sanctuary_DellaBianca', 0.807752251625061)]

*Madrid is to Spain, as Berlin is to ...*

In [13]:
wv.most_similar_cosmul(positive=['Spain','Berlin'],negative=['Madrid'])

[('Germany', 0.9708642959594727),
 ('Austria', 0.8568971753120422),
 ('German', 0.8524766564369202),
 ('Hungary', 0.8441289663314819),
 ('Poland', 0.8382856845855713),
 ('Annita_Kirsten', 0.8365224599838257),
 ('Thielert_AG_Hamburg', 0.8243475556373596),
 ('Buffalo_Sabres_Jochen_Hecht', 0.8205474019050598),
 ('symbol_RSTI', 0.8203197121620178),
 ('Saxony', 0.8148468732833862)]

También es posible encontrar palabras extrañas dentro de un grupo de palabras. 

Por ejemplo, ¿cuál de las siguientes palabras no encaja en la lista ``[Jupyter, Earth, Saturday, Mars, Moon]``?

In [ ]:
wv.doesnt_match(['Jupyter','Earth','Saturday','Mars','Moon'])

**Ejercicio** ¿Qué palabra no encaja en la siguiente lista ``[April, May, September, Monday, July]``?

## Glove y Fasttext

Además de word2vec, han aparecido otros embeddings. Los más conocidos son [Glove](https://nlp.stanford.edu/projects/glove/) y [FastText](https://fasttext.cc/). 

Como hemos explicado anteriormente, el método word2vec aprende la representación de las palabras mediante una red neuronal. En cambio estos otros dos embeddings funcionan de manera un poco distinta.

Glove se basa en técnicas de factorización de matrices. Para ello comienza construyendo una gran matriz con tantas filas y columnas como palabras. En esta matriz la entrada i,j indica el número de veces que la palabra i aparece en la misma frase que la palabra j. Seguidamente dicha matriz de co-ocurrencias se factoriza para producir una representación de baja dimensión.  

Tanto Glove como wor2vec tienen el problema de que no sirven para codificar palabras "raras" o que no aparecen en el vocabulario. Para resolver dicho problema surgió FastText. 

FastText es una extensión del modelo word2vec. En lugar de aprender vectores para cada palabra directamente, FastText representa cada palabra como un n-grama de caracteres. Por ejemplo, si tomamos la palabra artificial y usamos n=3, la representación de dicha palabra viene dada por <ar, art, rti, tif, ifi, fic, ici, ial, al> donde < y > indican respectivamente el principio y final de una palabra. 

Este método ayuda a capturar el signficado de palabras más cortas y permite comprender los sufijos y prefijos. Una vez que las palabras son partidas en n-gramas se entrena un modelo similar al de word2vec. Una ventaja de FastText es que funciona con palabras raras que no habían sido vistas anteriormente (cosa que no ocurría con los otros modelos). 



Desde el punto de vista de su uso no hay diferencias con respecto a word2vec. Vamos a comenzar descargando un modelo glove y otro modelo fasttext. 

In [ ]:
import gensim.downloader as api
wvGlove = api.load('glove-twitter-25')
wvFastText = api.load('fasttext-wiki-news-subwords-300')

**Ejercicio** ¿Qué otros modelos de fasttext y glove proporciona la librería gensym? Consulta lo que hace la función ``api.info()``.

**Ejercicio** Compara los resultados proporcionados por cada uno de los embeddings para los ejercicios presentados en el apartado anterior. Añade tantas celdas como necesites. 

## Otros idiomas

Los embeddings son particulares de cada idioma por lo que la representación de palabras en inglés no nos sirve cuando estamos trabajando en español. Afortunadamente, existen [modelos preentrenados para nuestro idioma](https://github.com/dccuchile/spanish-word-embeddings).

Su uso es relativamente sencillo. Debemos descargar el fichero de los pesos y cargarlo. Seguidamente podremos usarlo como hemos visto anteriormente. Por ejemplo vamos a cargar un modelo FastText. Comenzamos descargando el modelo.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec -O wiki.es.vec

A continuación mostramos las primeras filas del modelo, como se puede ver el documento que acabamos de descargar contiene en cada línea un n-grama (recordar qué es lo que hace FastText) y a continuación su representación en forma de vector. 

In [ ]:
!head wiki.es.vec

La carga del modelo es un poco distinta a lo visto anteriormente. En concreto debemos usar el siguiente código (notar que no cargamos todo el documento sino solo los 100000 primeros n-gramas). 

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'wiki.es.vec'
cantidad = 100000
wvFastTextSpanish = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

Ahora podemos utilizar las mismas funciones vistas anteriormente. 

**Ejercicio:** Encuentra las 5 palabras más similares a bicicleta. 

**Ejercicio:** Responde a la siguiente analogía. *Hombre es a actor como mujer es a...*

**Ejercicio:** Responde a la siguiente analogía. *Canta es a cantar como juega es a ...*

**Ejercicio:** Responde a la siguiente analogía. *Madrid es a España como Lisboa es a ...*

**Ejercicio:** Encuentra la palabra que no encaja en la lista ``[lunes, martes, septiembre, jueves, viernes]``

**Ejercicio:** Descarga alguno de los otros modelos proporcionados en el [zoo de modelos](https://github.com/dccuchile/spanish-word-embeddings) y encuentra un ejemplo de analogía y otro de palabra que no encaje usando dicho modelo (añade tantas celdas como necesites).

Recuerda guardar tu notebook en GitHub con la opción *Save in GitHub* del menú *File*.